<a href="https://colab.research.google.com/github/CODERZZZ123/StockPred_NewsHeadlines/blob/master/pred_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import nltk.corpus
import re
nltk.download('stopwords')
nltk.download('wordnet')


from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
# !pip install sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import euclidean_distances
# !pip install textattack
# !pip install --upgrade gensim --quiet

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# import gensim

# !pip install transformers --quiet

In [3]:
# import transformers

In [4]:
# !pip install sacremoses --quiet

In [5]:
# import sacremoses

In [6]:
# !pip install nlpaug --quiet

In [7]:
# import nlpaug.augmenter.char as na
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc
# from nlpaug.util import Action

In [8]:
# from nlpaug.util.file.download import DownloadUtil
# DownloadUtil.download_word2vec(dest_dir = ".")
# DownloadUtil.download_fasttext(dest_dir = "." , model_name = 'crawl-300d-2M')


In [9]:
# from google.colab import drive
# drive.mount('/content/drive')
# import requests
# import time

# while True:
#     try:
#         requests.get('https://www.google.com')
#         print("Kept alive.")
#     except:
#         print("Failed to keep alive.")
#     time.sleep(600)

In [10]:
from sklearn import  set_config
set_config(display = 'diagram')

In [11]:
df_NewsDjia = pd.read_csv('News_DJIA.csv', delimiter=',')
df_DjiaStock = pd.read_csv('DJIA_t.csv',delimiter=',')

In [12]:
df_DjiaStock

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234
...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609


In [13]:
for i in df_NewsDjia.columns :
    df_NewsDjia[i] = df_NewsDjia[i].astype(str)

In [14]:
# df_NewsDjia

In [15]:
num_columns = df_NewsDjia.columns.shape[0]
col_names = df_NewsDjia.columns.values.tolist()
col_names = col_names[1:]
df_NewsDjia.loc[:, 'merged'] = " "
for each_col_ind in range(num_columns-1):
    df_NewsDjia.loc[:, 'merged'] =  df_NewsDjia.loc[:, 'merged'] + " " + df_NewsDjia[col_names[each_col_ind]] + " "

In [16]:
df_finalised = df_NewsDjia[['merged']]

In [17]:
date_numric_stopwords = "THOUSAND MILLION BILLION TRILLION HUNDRED ANNUAL ANNUALLY ANNUM YEAR YEARLY QUARTER QUARTERLY QTR MONTH MONTHLY WEEK WEEKLY DAY DAILY JANUARY FEBRUARY MARCH APRIL MAY JUNE JULY AUGUST SEPTEMBER OCTOBER NOVEMBER DECEMBER JAN FEB MAR APR MAY JUN JUL AUG SEP SEPT OCT NOV DEC MONDAY TUESDAY WEDNESDAY THURSDAY FRIDAY SATURDAY SUNDAY TWO THREE FOUR FIVE SIX SEVEN EIGHT NINE TEN ELEVEN TWELVE THIRTEEN FOURTEEN FIFTEEN SIXTEEN SEVENTEEN EIGHTEEN NINETEEN TWENTY THIRTY FORTY FIFTY SIXTY SEVENTY EIGHTY NINETY FIRST SECOND THIRD FOURTH FIFTH SIXTH SEVENTH EIGHTH NINTH TENTH II III IV V VI VII VIII IX X XI XII XIII XIV XV XVI XVII XVIII XIX XX"
date_numric_stopwords = date_numric_stopwords.lower()
date_numric_stopwords = date_numric_stopwords.split(" ")

In [18]:

def remove_unecssary_word(sent):
    sent = sent.replace("b'",'')
    sent = sent.replace('b"','')
    sent = sent.lower()
    sent = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", sent)
    stop = stopwords.words('english')
    stop.extend(date_numric_stopwords)
    sent = " ".join([word for word in sent.split() if word not in (stop)])
    sent = " ".join([WordNetLemmatizer().lemmatize(word) for word in sent.split() ])
    return sent

In [19]:
df_finalised['merged'] = df_finalised['merged'].apply(remove_unecssary_word)

<ipython-input-19-6847205c14ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finalised['merged'] = df_finalised['merged'].apply(remove_unecssary_word)


In [20]:
df_finalised.iloc[0][0]

'0 georgia down russian warplane country move brink war breaking musharraf impeached russia today column troop roll south ossetia footage fighting youtube russian tank moving towards capital south ossetia reportedly completely destroyed georgian artillery fire afghan child raped impunity un official say sick old raped nothing 150 russian tank entered south ossetia whilst georgia shoot russian jet breaking georgia invades south ossetia russia warned would intervene so side enemy combatent trial nothing sham salim haman sentenced 5 12 year kept longer anyway feel like georgian troop retreat osettain capital presumably leaving several people killed video u prep georgia war russia rice give green light israel attack iran say u veto israeli military ops announcingclass action lawsuit behalf american public fbi sorussia georgia war nyts top story opening ceremony olympics fucking disgrace yet proof decline journalism china tell bush stay country affair world war start today georgia invades s

In [21]:
# from textattack.augmentation import EmbeddingAugmenter
# embed_aug = EmbeddingAugmenter()
# embed_aug.augment(df_finalised.iloc[0][0])

In [22]:
# from textattack.augmentation import WordNetAugmenter
# wordnet_aug = WordNetAugmenter()
# wordnet_aug.augment(df_finalised.iloc[0][0])

In [23]:
# from textattack.augmentation import EasyDataAugmenter
# eda_aug = EasyDataAugmenter()
# eda_aug.augment(df_finalised.iloc[0][0])

In [24]:
df_finalised = pd.concat([df_finalised,df_NewsDjia['Label']],axis = 1)

In [25]:
# Positive_word = ""
# with open('Positive_words.txt', 'r') as file:
#     Positive_word = file.read().replace('\n', '')
# Positive_word = " ".join([WordNetLemmatizer().lemmatize(word) for word in Positive_word.split(',') ])

In [26]:
# Negative_word = ""
# with open('Negative_words.txt','r') as file:
#     Negative_word = file.read().replace('\n','')
# Negative_word = " ".join([WordNetLemmatizer().lemmatize(word) for word in Negative_word.split(',') ])

In [27]:
# Positive_word = Positive_word.split(',')
# Negative_word = Negative_word.split(',')
# Negative_word = Negative_word[0].split()
# Positive_word = Positive_word[0].split()

In [28]:
# set_post = set(Positive_word)
# set_neg = set(Negative_word)
# def Count(obj):
#     count_post = 1
#     count_neg = 1
#     obj = obj.split()
#     for i in obj:
#         if i in set_post:
#             count_post = count_post + 1
#         elif i in set_neg:
#             count_neg = count_neg + 1
#     return count_post/count_neg


In [29]:
# df_finalised['ratio_pos_neg'] = df_finalised['merged'].apply(Count)
# ratio_column = df_finalised['ratio_pos_neg']
# ratio_column = np.array(ratio_column).reshape((-1,1))
X_data = df_finalised['merged']

In [30]:
Y_data = df_finalised['Label']
Y_data = Y_data.astype(int)

In [31]:
# from textattack.augmentation import EasyDataAugmenter
# eda_aug = EasyDataAugmenter()
# data_X = []
# label = []
# # open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt', 'w').close()
# # open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/label.txt','w').close()
# with open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt', 'a') as f1 , open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/label.txt','a') as f2:
#   for i in range(0,X_data.shape[0]):
#     data_X.append(X_data[i])
#     label.append(Y_data[i])
#     f1.write(X_data[i])
#     f1.write(",")
#     y = str(Y_data[i])
#     f2.write(y)
#     f2.write(",")
#     if i%10 == 0:
#       data = eda_aug.augment(X_data[i])
#       for d in data:
#         data_X.append(d)
#         label.append(Y_data[i])
#         f1.write(d)
#         f1.write(",")
#         f2.write(y)
#         f2.write(",")

#     print(i)

In [32]:
Y_data = np.array(Y_data)
Y_data

array([0, 1, 0, ..., 1, 1, 1])

In [33]:
# X_data[0]

In [34]:
# from google.colab import drive
# drive.mount('/content/drive')

In [35]:
# data_X

In [36]:
data_augmented = np.genfromtxt('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt',dtype = str , delimiter=",")

In [37]:
data_augmented.shape

(2864,)

In [38]:
count = 0
count2 = 0
for i in data_augmented:
    if len(i) != 0:
        if i[0] =='0' or i[0] == '1':
            if i[1] == " ":

              count = count + 1
    count2 = count2 + 1
print(count)
print(count2)

2788
2864


In [39]:
index = 0
for i in data_augmented:
  if len(i) != 0 :
    if i[0] != '1' and i[0] != '0' :
        data_augmented[index] = data_augmented[index-1][0] + " " + i
    index += 1

In [40]:
data_augmented.shape

(2864,)

In [41]:
data_augmented.shape

(2864,)

In [42]:
data_augmented = data_augmented[:-1]

In [43]:
label_updated = data_augmented.astype('<U1')

In [44]:
label_updated

array(['0', '0', '0', ..., '1', '1', '0'], dtype='<U1')

In [45]:
label_updated = label_updated.astype(np.float)

<ipython-input-45-39bc489429a0>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  label_updated = label_updated.astype(np.float)


In [46]:
data_augmented = list(data_augmented)

In [47]:
data_augmented = [i[2:] for i in data_augmented]

In [48]:
# data_augmented

In [49]:
X_data.shape

(1989,)

In [50]:
x_train , x_test , y_train, y_test = train_test_split(data_augmented,label_updated , test_size = 0.2 , random_state = 10)
# x_train , x_test , y_train , y_test = data_augmented[:2290] , data_augmented[2290:] , label_updated[:2290] , label_updated[2290:]

In [51]:
# tf_idf = TfidfVectorizer(min_df=10  )
# x_train_tf = tf_idf.fit_transform(x_train)
# x_test_tf = tf_idf.transform(x_test)
# x_train = x_train_tf.toarray()
# x_test = x_test_tf.toarray()

In [52]:
# mean_not_zero = x_train[x_train != 0].mean()
# x_train[x_train == 0 ] = x_train[x_train == 0] +  mean_not_zero
# x_test[x_test == 0 ] = x_test[x_test == 0] + mean_not_zero

In [53]:
ct_vect = CountVectorizer(ngram_range=(1,2) , min_df = 5)
x_train = ct_vect.fit_transform(x_train).toarray()
x_test = ct_vect.transform(x_test).toarray()

In [54]:
# from google.colab import drive
# drive.mount('/content/drive')

In [55]:
# scaler = StandardScaler()
# scaler.fit(x_train)
# x_train = scaler.transform(x_train)
# x_test = scaler.transform(x_test)

In [56]:
# pca = PCA()
# pca.fit(x_train)
# cumsum = np.cumsum(pca.explained_variance_ratio_)*100
# d = [n for n in range(len(cumsum))]
# plt.figure(figsize=(10, 10))
# plt.plot(d,cumsum, color = 'red',label='cumulative explained variance')
# plt.title('Cumulative Explained Variance as a Function of the Number of Components')
# plt.ylabel('Cumulative Explained variance')
# plt.xlabel('Principal components')
# plt.axhline(y = 90, color='k', linestyle='--', label = '95% Explained Variance')
# plt.legend(loc='best')

In [57]:
# # VARIANCE EXPLAINED 90%
# pca = PCA(0.90)
# pca.fit(x_train)
# x_train = pca.transform(x_train)
# x_test = pca.transform(x_test)
# pca.explained_variance_ratio_

In [58]:
# K = [int(i) for i in range(x_train.shape[0]//3)]
# param_grid = {
#     'n_neighbors' : K
# }
# knn = KNeighborsClassifier()
# grid_search = GridSearchCV(estimator=knn,scoring='accuracy',param_grid=param_grid,cv = 10,n_jobs = -1,verbose = 2)
# grid_search.fit(x_train,y_train)
# grid_search.best_estimator_
# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(x_train,y_train)
# y_pred = knn.predict(x_test)
# print(accuracy_score(y_pred,y_test))
# print(classification_report(y_pred,y_test))

In [59]:
# Naive bayes

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train,y_train)
# gnb.partial_fit(x_train,y_train,[0,1])
gnb.score(x_test,y_test)


0.6055846422338569

In [60]:
from sklearn.naive_bayes import MultinomialNB
mun = MultinomialNB(alpha = 1 , force_alpha='warn')
mun.fit(x_train,y_train)
mun.score(x_test, y_test)

0.5986038394415357

In [61]:
from sklearn.naive_bayes import ComplementNB
cmb = ComplementNB(alpha = 1 , force_alpha = 'warn')
cmb.fit(x_train,y_train)
cmb.score(x_test,y_test)

0.6020942408376964

In [62]:
# xcgvxcv

In [ ]:
C = [0.05 , 0.1 , 0.5  , 1 , 5 , 10 ,15 , 25]
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
gamma = ['scale','auto']
degree = [int(x) for x in range(1,2)]
param_grid = {
    'kernel' : kernel,
    'gamma' : gamma,
    'degree' : degree,
    'C' : C
}
svc = SVC()
grid_search = GridSearchCV(estimator=svc,scoring='accuracy',param_grid=param_grid,cv = 5,n_jobs = -1,verbose = 6)
grid_search.fit(x_train,y_train)
best_gird_search = grid_search.best_estimator_
from sklearn.metrics import accuracy_score
y_pred = best_gird_search.predict(x_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print(accuracy_score(y_train,best_gird_search.predict(x_train)))
Score = cross_val_score(best_gird_search,X_data,Y_data,cv = 5)
Score

Fitting 5 folds for each of 64 candidates, totalling 320 fits


In [ ]:
grid_search.best_params_

In [ ]:
from sklearn.metrics import accuracy_score
model = grid_search.best_params_
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy for the trained data is {}".format(accuracy_score(y_train,model.predict(x_train))))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("classification report {}".format(classification_report(y_test,y_pred)))
# Score = cross_val_score(model,X_data,Y_data,cv = 5)
# Score

In [ ]:
!pip install tensorflow

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# model = Sequential()
# model.add(Dense(12,input_shape = (x_train.shape[1],),activation = 'relu'))
# model.add(Dense(8,activation = 'relu'))
# model.add(Dense(1,activation = 'sigmoid'))
# model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.fit(x_train,y_train,epochs=200,batch_size=1000)
# predictions = (model.predict(x_test) > 0.5).astype(int)
# abs(predictions.T - y_test).sum()/y_test.shape[0]


In [ ]:
import matplotlib.pyplot as plt
score_test = []
score_train = []
scale = [i for i in range(1,200,5)]
for i in range(1 , 200 , 5):
  randomclassifier=RandomForestClassifier(n_estimators=i,criterion='entropy')
  randomclassifier.fit(x_train,y_train)
  predictions = randomclassifier.predict(x_test)
  predictions_train = randomclassifier.predict(x_train)
  score_test.append(accuracy_score(y_test,predictions))
  score_train.append(accuracy_score(y_train,predictions_train))
plt.plot(scale ,score_test )
plt.plot(scale ,score_train )
  # matrix=confusion_matrix(y_test,predictions)
# print(matrix)
# score=
# print(score)
# report=classification_report(y_test,predictions)
# print(report)